## Sample Notebook for making predictions of mlcsv_future data

In [44]:
import numpy as np
import pandas as pd

### Define stub method for predicting whether a future close approach is PHA or not


In [1]:
#def get_classifier_prediction():
    """
    Simulate prediction classifier model which returns randome value of 0 or 1. The probabilities for this 
    method are:
    
    Non PHA (Not Potentially hazerdous) = 0.90
    PHA (Potentially hazerdous) = 0.10
    
    0 -> asteroid is not PHA (potentially hazerdous)
    1 -> asertoid is PHA (potentially hazerdous)
    
    returns int: 0 or 1
    """
    #prediction = np.random.choice(np.arange(2), p=[0.8, 0.2])
    #return prediction

### Read Test Future data into DataFrame

In [2]:
mlcsv_future_df = pd.read_csv('../csv Data/test_neo_public_mlcsv_future.csv')

NameError: name 'pd' is not defined

In [48]:
mlcsv_future_df.head()

,spkid,full_name,pdes,h,orbit_id,epoch,epoch.mjd,epoch.cal,e,a,...,orbit_id_cad,jd,cd,dist,dist_min,dist_max,v_rel,v_inf,t_sigma_f,h_cad
0,2000433,433 Eros (A898 PA),433,10.43,JPL 659,2459600.5,59600,2022-01-21.0,0.2227,1.458,...,659,2.485544e+06,2093-01-31 15:47:00.000000,0.182464,0.182464,0.182464,5.974914,5.972469,< 00:01,10.43
1,2000887,887 Alinda (A918 AA),887,13.87,JPL 428,2459600.5,59600,2022-01-21.0,0.5705,2.473,...,429,2.483346e+06,2087-01-25 21:50:00.000000,0.166325,0.166321,0.166329,11.268948,11.267526,< 00:01,13.87
2,2001221,1221 Amor (1932 EA1),1221,17.39,JPL 110,2459600.5,59600,2022-01-21.0,0.4358,1.919,...,110,2.487064e+06,2097-03-31 04:56:00.000000,0.140528,0.140527,0.140528,7.564784,7.562277,< 00:01,17.39
3,2001566,1566 Icarus (1949 MA),1566,16.35,JPL 126,2459600.5,59600,2022-01-21.0,0.8269,1.078,...,126,2.484582e+06,2090-06-14 07:08:00.000000,0.043536,0.043529,0.043543,28.169786,28.167613,< 00:01,16.35
4,2001620,1620 Geographos (1951 RA),1620,15.29,JPL 649,2459600.5,59600,2022-01-21.0,0.3354,1.246,...,652,2.486124e+06,2094-09-03 10:34:00.000000,0.190024,0.190023,0.190025,16.694264,16.693424,< 00:01,15.29


In [76]:
# Create new dataframe with descriptive meta-data column for Visualization
descr_future_df = mlcsv_future_df.loc[:, ['spkid', 'full_name', 'pdes', 'class', 'first_obs', 'last_obs', 'n_obs_used']]
descr_future_df.head()

,spkid,full_name,pdes,class,first_obs,last_obs,n_obs_used
0,2000433,433 Eros (A898 PA),433,AMO,1893-10-29,2021-05-13,9130
1,2000887,887 Alinda (A918 AA),887,AMO,1918-02-03,2021-07-31,2624
2,2001221,1221 Amor (1932 EA1),1221,AMO,1932-03-12,2021-12-05,573
3,2001566,1566 Icarus (1949 MA),1566,APO,1949-07-01,2021-08-11,1078
4,2001620,1620 Geographos (1951 RA),1620,APO,1951-09-14,2022-01-01,4724


#### Define list of features for making future predictions
> **Note:** This list of features will be finalized once the ML model is trained and optimized

In [54]:
# Define feature columns to subset dataframe
feature_columns = ['h','e', 'a', 'q', 'i', 'om', 'w', 'ma', 'ad', 'n', 'tp', 'dist', 'dist_min', 'dist_max', 'v_rel', 'v_inf']

In [55]:
output_df = mlcsv_future_df.loc[:, feature_columns]

In [56]:
output_df.head()

,h,e,a,q,i,om,w,ma,ad,n,tp,dist,dist_min,dist_max,v_rel,v_inf
0,10.43,0.2227,1.458,1.133,10.83,304.30,178.90,246.90,1.78,0.5597,2459802.57,0.182464,0.182464,0.182464,5.974914,5.972469
1,13.87,0.5705,2.473,1.062,9.39,110.43,350.49,86.61,3.88,0.2534,2459258.73,0.166325,0.166321,0.166329,11.268948,11.267526
2,17.39,0.4358,1.919,1.082,11.88,171.32,26.64,261.04,2.75,0.3708,2459867.34,0.140528,0.140527,0.140528,7.564784,7.562277
3,16.35,0.8269,1.078,0.187,22.80,87.96,31.44,0.33,1.97,0.8805,2459600.12,0.043536,0.043529,0.043543,28.169786,28.167613
4,15.29,0.3354,1.246,0.828,13.34,337.18,276.95,300.49,1.66,0.7089,2459684.44,0.190024,0.190023,0.190025,16.694264,16.693424


In [140]:
# list of predictions
predictions = []
type(predictions)

list

In [141]:
# iterate through mlcsv future dataframe to get model preduction for each asteroid close approach  
for row in output_df.iterrows():
    prediction = get_classifier_prediction()
    predictions.append(prediction)

In [144]:
# Create Pandas Series from predictions list
prediction_series = pd.Series(predictions)

In [146]:
# Add new prediction column to dataframe
output_df['PHA_prediction'] = prediction_series

In [149]:
# Merge output dataframe with prediction column to the descr_future_df dataframe
new_output_df = pd.merge(descr_future_df, output_df, how='inner', left_index=True, right_index=True)

In [150]:
new_output_df['PHA_prediction'].value_counts()

0    393
1    107
Name: PHA_prediction, dtype: int64

In [151]:
new_output_df.head()

,spkid,full_name,pdes,class,first_obs,last_obs,n_obs_used,h,e,a,...,ma,ad,n,tp,dist,dist_min,dist_max,v_rel,v_inf,PHA_prediction
0,2000433,433 Eros (A898 PA),433,AMO,1893-10-29,2021-05-13,9130,10.43,0.2227,1.458,...,246.90,1.78,0.5597,2459802.57,0.182464,0.182464,0.182464,5.974914,5.972469,0
1,2000887,887 Alinda (A918 AA),887,AMO,1918-02-03,2021-07-31,2624,13.87,0.5705,2.473,...,86.61,3.88,0.2534,2459258.73,0.166325,0.166321,0.166329,11.268948,11.267526,0
2,2001221,1221 Amor (1932 EA1),1221,AMO,1932-03-12,2021-12-05,573,17.39,0.4358,1.919,...,261.04,2.75,0.3708,2459867.34,0.140528,0.140527,0.140528,7.564784,7.562277,0
3,2001566,1566 Icarus (1949 MA),1566,APO,1949-07-01,2021-08-11,1078,16.35,0.8269,1.078,...,0.33,1.97,0.8805,2459600.12,0.043536,0.043529,0.043543,28.169786,28.167613,0
4,2001620,1620 Geographos (1951 RA),1620,APO,1951-09-14,2022-01-01,4724,15.29,0.3354,1.246,...,300.49,1.66,0.7089,2459684.44,0.190024,0.190023,0.190025,16.694264,16.693424,0


In [152]:
new_output_df.to_csv('test_future_predictions_data.csv', index=False)